In [1]:
%%sh
pip install lightgbm

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import re

In [3]:
train = pd.read_csv('./data/train_preprocessed.csv').fillna(' ')
test = pd.read_csv('./data/test_preprocessed.csv').fillna(' ')
sub = pd.read_csv('./data/sample_submission.csv')
INPUT_COLUMN = "comment_text"
LABELS = train.columns[2:]

In [4]:
sub=pd.read_csv("outputs/cnn_glove_twitter_cv_3_submission.csv")
sub.head()
sub.shape

(153164, 7)

In [6]:
def get_subs(nums):
    subs = np.hstack([np.array(pd.read_csv("outputs/" + str(l) + "_submission.csv")[LABELS]) for l in sublabels])
    oofs = np.hstack([np.array(pd.read_csv("outputs/" + str(l) + "_oof.csv")[LABELS]) for l in sublabels])
    return subs, oofs

In [7]:
sublabels = ['cnn_glove_twitter_cv','bilstm_cnn_glove_twitter_cv']
subs, oofs = get_subs(sublabels)

In [8]:
X_train = oofs
X_test = subs

In [9]:
X_train.shape

(159571, 12)

In [10]:
X_test.shape

(153164, 12)

In [11]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score

stacker = lgb.LGBMClassifier(max_depth=3, metric="auc", n_estimators=125, num_leaves=10, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.45, colsample_bytree=0.45, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
    
# Fit and submit
scores = []
for label in LABELS:
    print(label)
    score = cross_val_score(stacker, X_train, train[label], cv=5, scoring='roc_auc')
    print("AUC:", score)
    scores.append(np.mean(score))
    stacker.fit(X_train, train[label])
    sub[label] = stacker.predict_proba(X_test)[:,1]
print("CV score:", np.mean(scores))

toxic
AUC: [0.9790311  0.97948162 0.98104419 0.97984363 0.97825441]
severe_toxic
AUC: [0.98991301 0.99044202 0.9888139  0.99057071 0.99028526]
obscene
AUC: [0.99246275 0.99275784 0.99373844 0.99253087 0.99265268]
threat
AUC: [0.9749159  0.98533355 0.9843935  0.98356276 0.98077007]
insult
AUC: [0.98564618 0.98533154 0.98637602 0.98603696 0.98461002]
identity_hate
AUC: [0.98657831 0.98406924 0.98664803 0.98767493 0.99130171]
CV score: 0.9861690382767573


In [12]:
sub.to_csv("outputs/submission_ensemble.csv", index=False)

In [13]:
sub.shape

(153164, 7)